In [1]:
import pandas as pd
import json
from scipy import stats
import numpy as np

## 등록 수 제한 전 등록된 상품

In [4]:
read_path = '../csv/pid_uid_191013_191116.csv'
before_product = pd.read_csv(read_path)

before_product

,register_date,pid,uid,device,bizlicense
0,2019-10-22,110387799,215,a,0
1,2019-10-17,110126623,215,a,0
2,2019-11-07,111330688,471,i,0
3,2019-11-01,110992357,719,a,0
4,2019-10-31,110940662,719,a,0
...,...,...,...,...,...
2072571,2019-11-16,111925303,10276452,a,0
2072572,2019-11-16,111925222,10276452,a,0
2072573,2019-11-16,111925149,10276452,a,0
2072574,2019-11-16,111925033,10276452,a,0


### 하루 50개 이상 등록했던 유저 리스트

In [12]:
before_day_pro = before_product[before_product['bizlicense'] == 1].groupby(['uid', 'register_date'], as_index=False).agg({'pid': 'count'})

before_pro_over = before_day_pro[before_day_pro['pid'] > 50]

before_pro_over

,uid,register_date,pid
594,22318,2019-11-14,67
957,38044,2019-10-20,65
967,38044,2019-11-11,60
2000,87148,2019-10-28,51
2011,87148,2019-11-09,58
...,...,...,...
120807,10178974,2019-11-15,62
120889,10192107,2019-11-04,75
121187,10241358,2019-11-16,55
121213,10243989,2019-11-14,87


In [14]:
before_pro_over_uid = before_pro_over.groupby(['uid'], as_index=False).agg({'pid': 'count'})

before_pro_over_uid.drop(['pid'], axis=1, inplace=True)

before_pro_over_uid

,uid
0,22318
1,38044
2,87148
3,112778
4,118553
...,...
334,10176725
335,10178974
336,10192107
337,10241358


### 하루 50개 이상 등록했던 유저들의 등록 상품

In [16]:
before_pro_over_product = pd.merge(before_product, before_pro_over_uid, on='uid')

before_pro_over_product

,register_date,pid,uid,device,bizlicense
0,2019-11-14,111786528,22318,w,1
1,2019-11-14,111786453,22318,w,1
2,2019-11-14,111786452,22318,w,1
3,2019-11-14,111786451,22318,w,1
4,2019-11-14,111786366,22318,w,1
...,...,...,...,...,...
203229,2019-11-13,111680504,10241358,w,1
203230,2019-11-13,111680391,10241358,w,1
203231,2019-11-13,111680136,10241358,w,1
203232,2019-11-13,111680071,10241358,w,1


# 등록 제한 전 등록된 상품의 상품명

In [17]:
read_path = '../csv/pid_name_191013_191116.csv'
before_pid_name = pd.read_csv(read_path)

before_pid_name

,pid,name
0,109865536,에이딕트 고체향수 3번사용
1,109865600,뉴발란스 벨로아 벨루아 벨벳
2,109868544,가내스라 홀가먼트니트
3,109870400,패딩머플러 목도리
4,109871680,스톤아일랜드 카고바지
...,...,...
2072569,111925439,모토로라 버브버즈200
2072570,111925503,[s]18fw 무스너클 발리스틱 블랙 백화점판 새상품
2072571,111925567,Y-3 카모
2072572,111925631,?롱패딩 7-8만원 팝니다 새거에요?


In [23]:
before_pid_name_over = pd.merge(before_pid_name, before_pro_over_product[['pid', 'uid']], on='pid')

before_pid_name_over

,pid,name,uid
0,109853504,MRR 19인치 st 검메탈 휠 판매 30만원,5623415
1,109854912,PRADA 프라다 스니커즈 MG58 38-44,9533857
2,109855552,✔EXR 크로스백,112778
3,109855808,도매가✔구찌 디오니소스 미니,3310884
4,109855872,❤️수입고퀄 무료배송❤️테그호이어 공유시계,1807834
...,...,...,...
203229,111921215,☆미러급☆ 구찌 오피디아 클러치백,7019680
203230,111921919,◇미러급◇ 스톤아일랜드 파이핑 숏패딩,7019680
203231,111922367,샤넬 가방,2875050
203232,111923711,[99% 동일] 루이비통 흑다미에 DISTRICT 크로스백,377896


In [28]:
abuse = before_pid_name_over.groupby(['uid', 'name'], as_index=False).agg({'pid': 'count'})

abuse_pid = abuse[abuse['pid'] > 1]

abuse_pid['pid'].sum()

95848